## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-07-13-03-44 +0000,nypost,Modern day ‘Noah’ hoping to save rare cat from...,https://nypost.com/2026/02/07/us-news/modern-d...
1,2026-02-07-13-02-44 +0000,bbc,US wants Russia and Ukraine to end war by June...,https://www.bbc.com/news/articles/cd0y0y9z231o...
2,2026-02-07-13-00-33 +0000,nyt,A Mosque Bombing Undercuts Pakistan’s Bid for ...,https://www.nytimes.com/2026/02/07/world/asia/...
3,2026-02-07-13-00-33 +0000,wapo,A postcard arrived: ‘If you’re reading this I’...,https://www.washingtonpost.com/lifestyle/2026/...
4,2026-02-07-13-00-03 +0000,nyt,Prediction Markets and Casinos Go to War Over ...,https://www.nytimes.com/2026/02/07/business/de...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2434/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
97,trump,38
59,epstein,20
48,new,18
100,iran,14
75,will,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
85,2026-02-07-03-07-00 +0000,wsj,A federal judge has ordered the Trump administ...,https://www.wsj.com/politics/policy/judge-bloc...,107
113,2026-02-07-01-00-00 +0000,wsj,President Trump chose a Federal Reserve chair ...,https://www.wsj.com/economy/central-banking/th...,86
86,2026-02-07-03-03-00 +0000,wsj,The Trump administration announced a new strat...,https://www.wsj.com/politics/national-security...,82
114,2026-02-07-00-52-00 +0000,wsj,President Trump posted a video including image...,https://www.wsj.com/politics/trump-obamas-apes...,76
185,2026-02-06-21-25-00 +0000,wsj,Hobbit-Inspired Startup Becomes First New Bank...,https://www.wsj.com/finance/banking/hobbit-ins...,74


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
85,107,2026-02-07-03-07-00 +0000,wsj,A federal judge has ordered the Trump administ...,https://www.wsj.com/politics/policy/judge-bloc...
275,61,2026-02-06-17-16-34 +0000,nyt,Milan Winter Olympics Opening Ceremony: What T...,https://www.nytimes.com/2026/02/06/world/europ...
23,51,2026-02-07-11-00-00 +0000,nypost,How Jeffrey Epstein got his hooks into ‘cheap’...,https://nypost.com/2026/02/07/world-news/newly...
197,47,2026-02-06-20-55-59 +0000,wapo,U.S. and Iran hold nuclear talks amid threats ...,https://www.washingtonpost.com/national-securi...
174,46,2026-02-06-21-44-00 +0000,wsj,Bill and Hillary Clinton are once again prepar...,https://www.wsj.com/politics/looming-epstein-d...
293,44,2026-02-06-16-12-52 +0000,nypost,Ashleigh Banfield stands by single sourced acc...,https://nypost.com/2026/02/06/media/ashleigh-b...
46,42,2026-02-07-10-00-00 +0000,wapo,"ICE isn’t at the Super Bowl, but Bad Bunny wil...",https://www.washingtonpost.com/immigration/202...
32,40,2026-02-07-10-30-00 +0000,wsj,"When considered as a percentage of GDP, the 20...",https://www.wsj.com/tech/ai/ai-spending-tech-c...
16,35,2026-02-07-12-00-00 +0000,missionlocal,People we meet: His wife told him to get out o...,https://missionlocal.org/2026/02/people-we-mee...
55,33,2026-02-07-09-00-00 +0000,cbc,Should you stay or should you go? Canadians re...,https://www.cbc.ca/news/world/cuba-travel-advi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
